In [ ]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

planets = NasaExoplanetArchive.query_criteria(
    table="pscomppars",
    select="hostname, discoverymethod, disc_facility",
    where="discoverymethod like 'Transit'"
).to_pandas()


planet_targets = planets[
    planets['disc_facility'].str.contains('TESS', na=False)
]['hostname'].unique().tolist()

print(len(planet_targets), "planet hosts found")


In [ ]:
from astroquery.mast import Catalogs
import random

try:
    tic_query = Catalogs.query_region(
        "00h42m44s +41d16m09s",  # Andromeda region (well-observed by TESS)
        radius=5,  
        catalog="TIC"
    )
    
    mask = (tic_query['Tmag'] > 8) & (tic_query['Tmag'] < 13)
    filtered_tic = tic_query[mask]
    
    all_tic_targets = [f"TIC {int(tic)}" for tic in filtered_tic['ID'][:600]]
    
    control_stars = [t for t in all_tic_targets if t not in planet_targets]
    
    random.seed(42)
    random.shuffle(control_stars)
    
    print(f"✓ {len(control_stars)} control stars available (will collect until 150+ successful)")
    
except Exception as e:
    print(f"Error querying TIC: {e}")
    print("Using fallback: selecting from generated TIC IDs...")
    control_stars = [f"TIC {i}" for i in range(100000000, 100000600)]
    random.seed(42)
    random.shuffle(control_stars)
    print(f"✓ {len(control_stars)} fallback control stars available")


In [ ]:
import lightkurve as lk
import os
import warnings
warnings.filterwarnings('ignore')

CACHE_DIR = "lc_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

def load_lightcurve(target):
    path = os.path.join(CACHE_DIR, target.replace(" ", "_").replace("-", "_") + ".fits")
    

    if os.path.exists(path):
        try:
            print(f"Loading {target} from cache...")
            lc = lk.read(path)
            return lc.remove_nans().remove_outliers().normalize()
        except Exception as e:
            print(f"Cache read failed for {target}: {e}. Re-downloading...")
            os.remove(path)  
    

    print(f"Searching for {target} in TESS archive...")
    

    search_results = None
    try:
        search_results = lk.search_lightcurve(target, mission='TESS', author='SPOC')
        if len(search_results) == 0:
            print(f"  No SPOC data, trying all authors...")
            search_results = lk.search_lightcurve(target, mission='TESS')
    except Exception as e:
        print(f"  Search error: {e}")
        return None
    
    if search_results is None or len(search_results) == 0:
        print(f"  ✗ No data found for {target}")
        return None
    
    print(f"  Found {len(search_results)} results")
    

    for attempt in range(3):
        try:
            print(f"  Downloading (attempt {attempt + 1}/3)...")
            lc = search_results[0].download()
            
            lc.to_fits(path, overwrite=True)
            print(f"  ✓ Successfully downloaded {target}")
            
            return lc.remove_nans().remove_outliers().normalize()
        except Exception as e:
            print(f"  Download attempt {attempt + 1} failed: {e}")
            if attempt == 2:
                print(f"  ✗ All download attempts failed for {target}")
                return None
    
    return None

In [ ]:
from wotan import flatten
import numpy as np

def detrend(lc):
    try:
        time = lc.time.value
        flux = lc.flux.value
        mask = np.isfinite(time) & np.isfinite(flux)
        time = time[mask]
        flux = flux[mask]
        
        if len(time) < 10:
            raise ValueError(f"Insufficient data points: {len(time)}")
        
        flat_flux, _ = flatten(time, flux, method='biweight', window_length=0.5, return_trend=True)
        return time, flat_flux
    except Exception as e:
        print(f"  Detrending error: {e}")
        raise

In [ ]:
from transitleastsquares import transitleastsquares
from scipy.stats import median_abs_deviation

def detect_tls(time, flux):
    try:
        model = transitleastsquares(time, flux)
        results = model.power(use_gpu=True)
        
        if not hasattr(results, 'period') or results.period is None:
            raise ValueError("TLS returned invalid results")
        
        return results
    except Exception as e:
        print(f"  TLS detection error: {e}")
        raise

def calculate_shape_features(time, flux, period, duration, t0):
    phase = ((time - t0) % period) / period
    phase[phase > 0.5] -= 1.0  # Center around 0
    
    # Get in-transit points
    in_transit = np.abs(phase) < (duration / period / 2)
    
    if np.sum(in_transit) < 5:
        return 0, 0, 0
    
    transit_flux = flux[in_transit]
    transit_phase = phase[in_transit]
    
    # 1. Transit shape symmetry (compare first half vs second half)
    mid_idx = len(transit_flux) // 2
    first_half = transit_flux[:mid_idx]
    second_half = transit_flux[mid_idx:]
    symmetry = np.std(first_half - second_half[::-1][:len(first_half)]) if len(first_half) > 0 else 0
    
    # 2. Ingress/egress duration ratio (V-shape detection)
    sorted_indices = np.argsort(transit_flux)
    deepest_point = np.median(sorted_indices[:max(1, len(sorted_indices)//5)])
    ingress_points = np.sum(transit_phase < 0)
    egress_points = np.sum(transit_phase > 0)
    shape_ratio = abs(ingress_points - egress_points) / max(ingress_points + egress_points, 1)
    
    # 3. Transit depth variability
    depth_std = np.std(transit_flux)
    
    return symmetry, shape_ratio, depth_std

def odd_even_test(time, flux, period, duration, t0):
    phase = ((time - t0) % period) / period
    phase[phase > 0.5] -= 1.0
    
    # Identify transits
    in_transit = np.abs(phase) < (duration / period / 2)
    transit_number = np.floor((time - t0) / period)
    
    # Separate odd and even transits
    odd_mask = in_transit & (transit_number % 2 == 1)
    even_mask = in_transit & (transit_number % 2 == 0)
    
    if np.sum(odd_mask) < 3 or np.sum(even_mask) < 3:
        return 0, 0, 0
    
    odd_flux = flux[odd_mask]
    even_flux = flux[even_mask]
    
    # 1. Depth difference
    odd_depth = 1 - np.median(odd_flux)
    even_depth = 1 - np.median(even_flux)
    depth_diff = abs(odd_depth - even_depth)
    
    # 2. Duration difference (check if one set is systematically longer)
    odd_duration = np.sum(odd_mask) / len(time) * period
    even_duration = np.sum(even_mask) / len(time) * period
    duration_diff = abs(odd_duration - even_duration) / max(duration, 1e-10)  # Avoid division by zero
    
    # 3. Shape consistency (MAD of transit depths) - Fixed to avoid inf
    mad_odd = median_abs_deviation(odd_flux)
    mad_even = median_abs_deviation(even_flux)
    
    # Safely compute MAD ratio with bounds to prevent inf
    if mad_even > 1e-10 and mad_odd > 1e-10:
        mad_ratio = mad_odd / mad_even
        # Cap the ratio to prevent extreme values
        mad_ratio = np.clip(mad_ratio, 0.01, 100)
        mad_ratio = abs(np.log(mad_ratio))
    else:
        mad_ratio = 0
    
    return depth_diff, duration_diff, mad_ratio

def check_multi_sector(target):
    try:
        search_results = lk.search_lightcurve(target, mission='TESS')
        if len(search_results) > 0:
            # Count unique sectors
            sectors = set()
            for i in range(len(search_results)):
                if hasattr(search_results[i], 'mission') and hasattr(search_results[i], 'sequence_number'):
                    sectors.add(search_results[i].sequence_number)
            return len(sectors)
        return 0
    except:
        return 0


In [ ]:
import torch

if torch.cuda.is_available():
    print("✓ CUDA is available. Using GPU for processing.")
else:
    print("✗ CUDA not available. Using CPU for processing.")

In [ ]:
!pip install torch torchvision torchaudio
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com

In [ ]:
def extract_features(target):
    print(f"\n{'='*60}")
    print(f"Analyzing: {target}")
    print(f"{'='*60}")
    
    def to_scalar(value):
        if value is None:
            return 0.0
        if isinstance(value, (list, np.ndarray)):
            val = float(value[0]) if len(value) > 0 else 0.0
        else:
            val = float(value)
        
        if not np.isfinite(val):
            return 0.0
        return val
    
    try:
        lc = load_lightcurve(target)
        if lc is None:
            print(f"✗ Failed to load lightcurve for {target}")
            return None
        
        print(f"  ✓ Lightcurve loaded: {len(lc)} data points")
        
        # Step 1: Multi-sector check
        num_sectors = check_multi_sector(target)
        print(f"  ✓ Available sectors: {num_sectors}")
        
        # Step 2: Detrend
        time, flux = detrend(lc)
        print(f"  ✓ Detrended: {len(time)} clean data points")
        
        # Step 3: TLS detection (get full results object)
        results = detect_tls(time, flux)
        period = to_scalar(results.period)
        duration = to_scalar(results.duration)
        depth = to_scalar(results.depth)
        snr = to_scalar(results.SDE)
        t0 = to_scalar(results.T0)
        
        print(f"  ✓ TLS complete: Period={period:.2f}d, Depth={depth:.4f}, SNR={snr:.2f}")
        
        # Step 4: SDE threshold check
        sde_pass = 1 if snr >= 7.0 else 0  # Standard threshold
        print(f"  ✓ SDE threshold (7.0): {'PASS' if sde_pass else 'FAIL'} (SNR={snr:.2f})")
        
        # Step 5: TLS model fit metrics
        rp_rs = to_scalar(results.rp_rs if hasattr(results, 'rp_rs') else 0)
        snr_pink = to_scalar(results.snr_pink_per_transit if hasattr(results, 'snr_pink_per_transit') else snr)
        odd_even_mismatch = to_scalar(results.odd_even_mismatch if hasattr(results, 'odd_even_mismatch') else 0)
        
        print(f"  ✓ Model fit: Rp/Rs={rp_rs:.4f}, SNR_pink={snr_pink:.2f}")
        
        # Step 6: Shape features
        symmetry, shape_ratio, depth_std = calculate_shape_features(time, flux, period, duration, t0)
        symmetry = to_scalar(symmetry)
        shape_ratio = to_scalar(shape_ratio)
        depth_std = to_scalar(depth_std)
        print(f"  ✓ Shape features: symmetry={symmetry:.6f}, ratio={shape_ratio:.4f}")
        
        # Step 7: Enhanced odd-even test
        depth_diff, duration_diff, mad_ratio = odd_even_test(time, flux, period, duration, t0)
        depth_diff = to_scalar(depth_diff)
        duration_diff = to_scalar(duration_diff)
        mad_ratio = to_scalar(mad_ratio)
        print(f"  ✓ Odd-even test: depth_diff={depth_diff:.6f}, dur_diff={duration_diff:.4f}")
        

        features = [
            period,           # 0: Orbital period
            depth,            # 1: Transit depth
            duration,         # 2: Transit duration
            snr,              # 3: Signal-to-noise ratio
            sde_pass,         # 4: SDE threshold pass/fail
            rp_rs,            # 5: Planet-to-star radius ratio
            snr_pink,         # 6: SNR accounting for red noise
            odd_even_mismatch,# 7: TLS odd-even mismatch
            symmetry,         # 8: Transit symmetry
            shape_ratio,      # 9: Ingress/egress ratio
            depth_std,        # 10: Transit depth variability
            depth_diff,       # 11: Odd-even depth difference
            duration_diff,    # 12: Odd-even duration difference
            mad_ratio,        # 13: Odd-even MAD ratio
            num_sectors,      # 14: Number of sectors
            len(time)         # 15: Total data points
        ]
        

        for i, feat in enumerate(features):
            if not np.isfinite(feat):
                print(f"  ⚠ Warning: Feature {i} is not finite ({feat}), setting to 0.0")
                features[i] = 0.0
        
        print(f"  ✓ Extracted {len(features)} features successfully")
        return features

    except Exception as e:
        print(f"  ✗ Feature extraction failed: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return None


In [ ]:

import random
random.seed(42)

planet_sample = random.sample(planet_targets, min(250, len(planet_targets)))

X, y = [], []
failed_targets = []

print("\n" + "="*60)
print(f"COLLECTING PLANET CANDIDATES ({len(planet_sample)} targets sampled)")
print("="*60)

for star in planet_sample:
    feat = extract_features(star)
    if feat:
        X.append(feat)
        y.append(1)
    else:
        failed_targets.append((star, "planet"))

planet_count = sum(y)
print(f"\n✓ Successfully collected {planet_count} planet samples")

print("\n" + "="*60)
print(f"COLLECTING NON-PLANET STARS ({len(control_stars)} targets sampled)")
print("="*60)


target_control_count = max(150, planet_count)

for star in control_stars:
    feat = extract_features(star)
    if feat:
        X.append(feat)
        y.append(0)
        current_control_count = len(y) - sum(y)
        if current_control_count >= target_control_count:
            print(f"\n✓ Reached target of {target_control_count} non-planets, stopping collection")
            break
    else:
        failed_targets.append((star, "control"))


print("\n" + "="*60)
print("CLEANING DATA")
print("="*60)
X_clean, y_clean = [], []
removed_count = 0

for i, (features, label) in enumerate(zip(X, y)):
    if all(np.isfinite(f) for f in features):
        X_clean.append(features)
        y_clean.append(label)
    else:
        removed_count += 1
        print(f"  Removed sample {i}: contains inf/nan")

X, y = X_clean, y_clean

print("\n" + "="*60)
print("COLLECTION SUMMARY")
print("="*60)
print(f"✓ Successfully collected: {len(X)} samples")
print(f"  - Planets: {sum(y)}")
print(f"  - Non-planets: {len(y) - sum(y)}")
print(f"  - Removed (inf/nan): {removed_count}")
print(f"  - Balance ratio: {max(sum(y), len(y)-sum(y)) / min(sum(y), len(y)-sum(y)):.2f}:1")

if failed_targets:
    print(f"\n✗ Failed to process {len(failed_targets)} targets:")
    for target, category in failed_targets[:10]:  # Show first 10
        print(f"  - {target} ({category})")
    if len(failed_targets) > 10:
        print(f"  ... and {len(failed_targets) - 10} more")


if sum(y) < 100:
    print(f"\n⚠ WARNING: Only {sum(y)} planets collected (target: 100+)")
if len(y) - sum(y) < 100:
    print(f"\n⚠ WARNING: Only {len(y) - sum(y)} non-planets collected (target: 100+)")

if len(X) < 200:
    print("\n⚠ WARNING: Dataset has fewer than 200 samples! Model may not be reliable.")
elif len(X) < 300:
    print(f"\n⚡ Dataset size: {len(X)} samples (minimum met, but target is 500+)")
elif len(X) < 500:
    print(f"\n⚡ Dataset size: {len(X)} samples (good, but target is 500+)")
else:
    print(f"\n✓ Dataset ready for training! ({len(X)} samples - EXCELLENT!)")


In [ ]:
import numpy as np
from collections import Counter
from cuml.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

X = np.array(X)
y = np.array(y)

print("\n" + "="*60)
print("CLASS BALANCE CHECK")
print("="*60)
class_counts = Counter(y)
print(f"Class distribution: {dict(class_counts)}")
print(f"  - Non-planets (0): {class_counts[0]} ({class_counts[0]/len(y)*100:.1f}%)")
print(f"  - Planets (1): {class_counts[1]} ({class_counts[1]/len(y)*100:.1f}%)")


imbalance_ratio = max(class_counts.values()) / min(class_counts.values())
print(f"  - Imbalance ratio: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 2:
    print("  ⚠ Significant class imbalance detected! Using class weights...")
    use_class_weights = True
else:
    print("  ✓ Classes are balanced")
    use_class_weights = False


print("\n" + "="*60)
print("TRAIN-VALIDATION-TEST SPLIT (70%-15%-15%)")
print("="*60)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# Split the 30% temp into 50/50 for validation and test (15% each of total)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Training set: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"  - Planets: {sum(y_train)} ({sum(y_train)/len(y_train)*100:.1f}%)")
print(f"  - Non-planets: {len(y_train) - sum(y_train)} ({(len(y_train)-sum(y_train))/len(y_train)*100:.1f}%)")

print(f"Validation set: {len(X_val)} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"  - Planets: {sum(y_val)} ({sum(y_val)/len(y_val)*100:.1f}%)")
print(f"  - Non-planets: {len(y_val) - sum(y_val)} ({(len(y_val)-sum(y_val))/len(y_val)*100:.1f}%)")

print(f"Test set: {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")
print(f"  - Planets: {sum(y_test)} ({sum(y_test)/len(y_test)*100:.1f}%)")
print(f"  - Non-planets: {len(y_test) - sum(y_test)} ({(len(y_test)-sum(y_test))/len(y_test)*100:.1f}%)")


if use_class_weights:
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(f"\nClass weights: {class_weight_dict}")
else:
    class_weight_dict = None


print("\n" + "="*60)
print("MODEL TRAINING")
print("="*60)
model = RandomForestClassifier(
    n_estimators=200, 
    random_state=42,
    class_weight=class_weight_dict,
)

print("Training Random Forest...")
model.fit(X_train, y_train)

# Evaluate on all three sets
train_accuracy = model.score(X_train, y_train)
val_accuracy = model.score(X_val, y_val)
test_accuracy = model.score(X_test, y_test)

print(f"\n✓ Model trained successfully!")
print(f"  Training accuracy:   {train_accuracy:.2%}")
print(f"  Validation accuracy: {val_accuracy:.2%}")
print(f"  Test accuracy:       {test_accuracy:.2%}")

# Check for overfitting
train_val_gap = train_accuracy - val_accuracy
print(f"\nGeneralization check:")
print(f"  Train-Val gap: {train_val_gap*100:.1f}%")
if train_val_gap > 0.15:
    print(f"  ⚠ Warning: Possible overfitting!")
elif train_val_gap > 0.05:
    print(f"  ⚡ Minor overfitting detected")
else:
    print(f"  ✓ Good generalization")


In [ ]:
# Feature names and importance analysis
import pandas as pd
import matplotlib.pyplot as plt

feature_names = [
    'Period',           # 0: Orbital period (days)
    'Depth',            # 1: Transit depth
    'Duration',         # 2: Transit duration (days)
    'SNR',              # 3: Signal-to-noise ratio (SDE)
    'SDE_Pass',         # 4: SDE threshold pass (1) or fail (0)
    'Rp_Rs',            # 5: Planet-to-star radius ratio
    'SNR_Pink',         # 6: SNR accounting for pink noise
    'OE_Mismatch',      # 7: TLS odd-even mismatch
    'Symmetry',         # 8: Transit symmetry
    'Shape_Ratio',      # 9: Ingress/egress ratio
    'Depth_Std',        # 10: Depth standard deviation
    'OE_Depth_Diff',    # 11: Odd-even depth difference
    'OE_Duration_Diff', # 12: Odd-even duration difference
    'OE_MAD_Ratio',     # 13: Odd-even MAD ratio
    'N_Sectors',        # 14: Number of TESS sectors
    'N_Points'          # 15: Total data points
]

print("\n" + "="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Get feature importances
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print("-" * 60)
for idx, row in feature_importance_df.head(10).iterrows():
    bar = '█' * int(row['importance'] * 100)
    print(f"{row['feature']:20s} │ {bar} {row['importance']:.4f}")

# Visualize feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['feature'][:10], feature_importance_df['importance'][:10])
plt.xlabel('Importance', fontsize=12)
plt.title('Top 10 Feature Importances - Random Forest', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
print(f"\n✓ Feature importance plot saved to: feature_importance.png")
plt.show()

# Check if model is learning astrophysics or artifacts
print("\n" + "="*60)
print("FEATURE QUALITY CHECK")
print("="*60)
top_features = feature_importance_df.head(3)['feature'].tolist()
physics_features = ['SNR', 'Depth', 'Period', 'Duration', 'SNR_Pink', 'Rp_Rs', 'SDE_Pass']
metadata_features = ['N_Sectors', 'N_Points']

physics_in_top = sum(1 for f in top_features if f in physics_features)
metadata_in_top = sum(1 for f in top_features if f in metadata_features)

if metadata_in_top >= 2:
    print("⚠ WARNING: Model may be learning observational bias!")
    print(f"  Top features include: {', '.join(top_features)}")
    print("  Consider removing N_Sectors and N_Points from features.")
elif physics_in_top >= 2:
    print("✓ Model is learning physical features (good!)")
    print(f"  Top features: {', '.join(top_features)}")
else:
    print("⚡ Model is learning shape/consistency features")
    print(f"  Top features: {', '.join(top_features)}")

In [ ]:
# Enhanced evaluation metrics and cross-validation
from sklearn.metrics import (classification_report, confusion_matrix, 
                            roc_auc_score, roc_curve, RocCurveDisplay,
                            precision_recall_curve, average_precision_score)
from sklearn.model_selection import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt

print("\n" + "="*60)
print("CROSS-VALIDATION")
print("="*60)

# 5-fold stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
cv_f1_scores = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1)

print(f"5-Fold Cross-Validation Results:")
print(f"  Accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
print(f"  F1 Score: {cv_f1_scores.mean():.3f} ± {cv_f1_scores.std():.3f}")
print(f"  Individual folds (accuracy): {[f'{s:.3f}' for s in cv_scores]}")

# Check CV stability
if cv_scores.std() < 0.03:
    print(f"  ✓ STABLE: CV std ({cv_scores.std():.3f}) < 0.03")
else:
    print(f"  ⚠ UNSTABLE: CV std ({cv_scores.std():.3f}) >= 0.03")

print("\n" + "="*60)
print("VALIDATION SET EVALUATION")
print("="*60)


y_val_pred = model.predict(X_val)
y_val_proba = model.predict_proba(X_val)[:, 1]

# Confusion Matrix for validation
cm_val = confusion_matrix(y_val, y_val_pred)
tn_val, fp_val, fn_val, tp_val = cm_val.ravel()
recall_val = tp_val / (tp_val + fn_val) if (tp_val + fn_val) > 0 else 0
specificity_val = tn_val / (tn_val + fp_val) if (tn_val + fp_val) > 0 else 0

print(f"Validation Set Metrics:")
print(f"  Accuracy:     {val_accuracy:.3f}")
print(f"  Recall:       {recall_val:.3f} {'✓ PASS' if recall_val > 0.7 else '✗ FAIL'} (target: > 0.70)")
print(f"  Specificity:  {specificity_val:.3f} {'✓ PASS' if specificity_val > 0.8 else '✗ FAIL'} (target: > 0.80)")

print("\n" + "="*60)
print("TEST SET EVALUATION")
print("="*60)


y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]


print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Non-Planet', 'Planet'], digits=3))


print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(f"                 Predicted")
print(f"                 Non-Planet  Planet")
print(f"Actual Non-Planet    {cm[0,0]:3d}        {cm[0,1]:3d}")
print(f"Actual Planet        {cm[1,0]:3d}        {cm[1,1]:3d}")


tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0

print(f"\nDetailed Metrics:")
print(f"  True Positives:  {tp} (correctly identified planets)")
print(f"  True Negatives:  {tn} (correctly identified non-planets)")
print(f"  False Positives: {fp} (false alarms)")
print(f"  False Negatives: {fn} (missed planets)")
print(f"  Recall (Sensitivity): {sensitivity:.3f} {'✓ PASS' if sensitivity > 0.7 else '✗ FAIL'} (target: > 0.70)")
print(f"  Specificity:          {specificity:.3f} {'✓ PASS' if specificity > 0.8 else '✗ FAIL'} (target: > 0.80)")

# ROC AUC Score
try:
    roc_auc = roc_auc_score(y_test, y_proba)
    print(f"  ROC AUC Score: {roc_auc:.3f}")
except:
    print("  ROC AUC Score: N/A (insufficient data)")
    roc_auc = None

# Average Precision (better for imbalanced datasets)
try:
    avg_precision = average_precision_score(y_test, y_proba)
    print(f"  Average Precision: {avg_precision:.3f}")
except:
    print("  Average Precision: N/A")
    avg_precision = None

# Plot ROC and Precision-Recall curves
if roc_auc is not None and len(np.unique(y_test)) > 1:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # ROC Curve
    RocCurveDisplay.from_estimator(model, X_test, y_test, ax=ax1)
    ax1.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
    ax1.set_title(f'ROC Curve (AUC = {roc_auc:.3f})', fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Precision-Recall Curve
    from sklearn.metrics import PrecisionRecallDisplay
    PrecisionRecallDisplay.from_estimator(model, X_test, y_test, ax=ax2)
    ax2.set_title(f'Precision-Recall Curve (AP = {avg_precision:.3f})', fontweight='bold')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('evaluation_curves.png', dpi=150, bbox_inches='tight')
    print(f"\n✓ Evaluation curves saved to: evaluation_curves.png")
    plt.show()

print("\n" + "="*60)
print("MODEL PERFORMANCE SUMMARY")
print("="*60)
print(f"✓ Random Forest with {model.n_estimators} trees")
print(f"✓ Trained on {len(X_train)} samples")
print(f"✓ Validated on {len(X_val)} samples")
print(f"✓ Tested on {len(X_test)} samples")
print(f"\nAccuracy:")
print(f"  Train: {train_accuracy:.2%}")
print(f"  Val:   {val_accuracy:.2%}")
print(f"  Test:  {test_accuracy:.2%}")
print(f"\nCross-Validation:")
print(f"  Mean: {cv_scores.mean():.2%} ± {cv_scores.std():.2%}")
if roc_auc:
    print(f"\nROC AUC: {roc_auc:.3f}")

print("\n" + "="*60)
print("REQUIREMENTS CHECK")
print("="*60)
print(f"Dataset Size:    {len(X)} samples {'✓' if len(X) >= 200 else '✗'} (minimum: 200)")
print(f"  Planets:       {sum(y)} {'✓' if sum(y) >= 100 else '✗'} (minimum: 100)")
print(f"  Non-planets:   {len(y)-sum(y)} {'✓' if len(y)-sum(y) >= 100 else '✗'} (minimum: 100)")
print(f"Recall (Test):   {sensitivity:.3f} {'✓ PASS' if sensitivity > 0.7 else '✗ FAIL'} (target: > 0.70)")
print(f"Specificity:     {specificity:.3f} {'✓ PASS' if specificity > 0.8 else '✗ FAIL'} (target: > 0.80)")
print(f"CV Stability:    {cv_scores.std():.3f} {'✓ PASS' if cv_scores.std() < 0.03 else '✗ FAIL'} (target: < 0.03)")

all_pass = (len(X) >= 200 and sum(y) >= 100 and (len(y)-sum(y)) >= 100 and 
            sensitivity > 0.7 and specificity > 0.8 and cv_scores.std() < 0.03)

if all_pass:
    print("\n🎉 ALL REQUIREMENTS MET! Model is ready for production.")
else:
    print("\n⚠️  Some requirements not met. Consider:")
    if len(X) < 200 or sum(y) < 100 or (len(y)-sum(y)) < 100:
        print("  - Collecting more data samples")
    if sensitivity <= 0.7:
        print("  - Adjust class weights to improve recall")
    if specificity <= 0.8:
        print("  - Review false positive cases")
    if cv_scores.std() >= 0.03:
        print("  - Increase dataset size for more stable CV")


In [ ]:
import joblib
from datetime import datetime

print("\n" + "="*60)
print("MODEL PERSISTENCE")
print("="*60)


model_package = {
    'model': model,
    'feature_names': feature_names,
    'training_metadata': {
        'training_date': datetime.now().isoformat(),
        'n_train_samples': len(X_train),
        'n_val_samples': len(X_val),
        'n_test_samples': len(X_test),
        'n_features': len(feature_names),
        'train_accuracy': float(train_accuracy),
        'val_accuracy': float(val_accuracy),
        'test_accuracy': float(test_accuracy),
        'cv_accuracy_mean': float(cv_scores.mean()),
        'cv_accuracy_std': float(cv_scores.std()),
        'test_recall': float(sensitivity),
        'test_specificity': float(specificity),
        'class_distribution': dict(Counter(y)),
        'hyperparameters': model.get_params(),
        'used_class_weights': use_class_weights
    },
    'feature_importances': dict(zip(feature_names, model.feature_importances_))
}


model_filename = 'exoplanet_model_v2.pkl'
joblib.dump(model_package, model_filename)

print(f"✓ Model saved to: {model_filename}")
print(f"  Model type: Random Forest")
print(f"  Number of trees: {model.n_estimators}")
print(f"  Training samples: {len(X_train)}")
print(f"  Validation samples: {len(X_val)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Test accuracy: {test_accuracy:.2%}")
print(f"  Test recall: {sensitivity:.2%}")
print(f"  Test specificity: {specificity:.2%}")
print(f"  File size: {os.path.getsize(model_filename) / 1024:.1f} KB")

print("\nTo load the model later:")
print("  loaded = joblib.load('exoplanet_model_v2.pkl')")
print("  model = loaded['model']")
print("  feature_names = loaded['feature_names']")

print("\n" + "="*60)
print("✓ PIPELINE UPDATED SUCCESSFULLY!")
print("="*60)
print("\nImplemented improvements:")
print("  ✓ 70%-15%-15% train-validation-test split")
print("  ✓ Balanced classes (1:1 ratio target)")
print("  ✓ Minimum 200 samples (100 planets + 100 non-planets)")
print("  ✓ Performance targets: Recall > 0.7, Specificity > 0.8")
print("  ✓ CV stability check: std < 0.03")
print("  ✓ Comprehensive validation set evaluation")
print("\nModel is ready for production use!")


In [ ]:
def predict_star(target, save_to_csv=True):
    from pathlib import Path
    
    print(f"\n{'='*70}")
    print(f"PREDICTION FOR: {target}")
    print(f"{'='*70}")
    
    features = extract_features(target)
    
    if features is None:
        print(f"\n✗ Could not make prediction for {target} (feature extraction failed)")
        return None
    
    features_array = np.array(features).reshape(1, -1)
    prediction = model.predict(features_array)[0]
    probability = model.predict_proba(features_array)[0][1]  # Probability of being a planet
    
    result = "Planet" if prediction == 1 else "Non-Planet"
    
    print(f"\n{'='*70}")
    print(f"PREDICTION RESULT: {result}")
    print(f"Planet Probability: {probability:.2%}")
    print(f"{'='*70}")
    
    if save_to_csv:
        import pandas as pd
        from datetime import datetime
        
        csv_file = 'predictions.csv'
        
        record = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'target_name': target,
            'prediction': result,
            'planet_probability': probability
        }
        
        for i, name in enumerate(feature_names):
            record[name.lower().replace(' ', '_')] = features[i]
        
        df_new = pd.DataFrame([record])
        
        try:
            if Path(csv_file).exists():
                df_existing = pd.read_csv(csv_file)
                df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            else:
                df_combined = df_new
            
            df_combined.to_csv(csv_file, index=False)
            print(f"✓ Prediction saved to {csv_file}")
        except Exception as e:
            print(f"✗ Failed to save to CSV: {e}")
    
    return {'target': target, 'prediction': result, 'probability': probability, 'features': features}

In [ ]:
print("Testing the model on new targets:")
print("="*60)

predict_star("TOI-270")

print()

predict_star("TIC 38846515")

In [ ]:
test_features = extract_features("TIC 307210830")
if test_features:
    print(f"\n\nExtracted {len(test_features)} features:")
    print(f"  Period: {test_features[0]:.2f} days")
    print(f"  Depth: {test_features[1]:.6f}")
    print(f"  Duration: {test_features[2]:.4f} days")
    print(f"  SNR: {test_features[3]:.2f}")
    print(f"  SDE Pass: {test_features[4]}")

In [ ]:
test_targets = ["TIC 16740101","TIC 188350478","TIC 38964113","TIC 377909730"]

print("Running batch predictions...")
print("="*60)

for target in test_targets:
    predict_star(target)
    print()

print("\n" + "="*60)
print("PREDICTIONS SUMMARY")
print("="*60)
print(f"\n✓ All predictions saved to: predictions.csv")

In [ ]:
predict_star("Ross 176")